In [ ]:
%%capture

!git clone https://github.com/hudeven/text
!git clone https://github.com/facebookresearch/pytext


In [ ]:
%%capture

!pip install --upgrade --force-reinstall ./text/stl_text/dataframes/datasets;
!pip install --upgrade --force-reinstall ./text;

!pip install --upgrade --force-reinstall ./pytext;
!pip install --upgrade --force-reinstall tensorboard;

!pip install --upgrade --force-reinstall --pre torch torchtext -f https://download.pytorch.org/whl/nightly/cu101/torch_nightly.html;
!pip install --upgrade --force-reinstall datasets;

!pip install pytorch-lightning

In [1]:
import os

import torch
from pytorch_lightning import Trainer
from stl_text.ops.utils.arrow import convert_csv_to_arrow
from stl_text.datamodule import DocClassificationDataModule
from task import DocClassificationTask

In [2]:
# convert csv to arrow format (only required for the first time)
data_path = "./glue_sst2_tiny"
for split in ("train.tsv", "valid.tsv", "test.tsv"):
    split_path = os.path.join(data_path, split)
    convert_csv_to_arrow(split_path)

# setup datamodule
datamodule = DocClassificationDataModule(data_path=data_path, batch_size=8, drop_last=True)
datamodule.setup("fit")

converted to arrow and saved to ./glue_sst2_tiny/train
converted to arrow and saved to ./glue_sst2_tiny/valid
converted to arrow and saved to ./glue_sst2_tiny/test
Loading cached processed dataset at ./glue_sst2_tiny/train/cache-f1e83dde37d3c060.arrow
Loading cached processed dataset at ./glue_sst2_tiny/train/cache-e464112f4fa64676.arrow
Loading cached processed dataset at ./glue_sst2_tiny/train/cache-3301217692ea7b7c.arrow
Loading cached processed dataset at ./glue_sst2_tiny/valid/cache-d7e81757a0546a02.arrow
Loading cached processed dataset at ./glue_sst2_tiny/valid/cache-daf7dc6f6a37bdc1.arrow
Loading cached processed dataset at ./glue_sst2_tiny/valid/cache-a1235ce082fa5b2a.arrow
Loading cached processed dataset at ./glue_sst2_tiny/test/cache-4a8ddec11e8e0938.arrow
Loading cached processed dataset at ./glue_sst2_tiny/test/cache-a6e5e3e4a7b8c81f.arrow
Loading cached processed dataset at ./glue_sst2_tiny/test/cache-c33ac2d9284c0aa3.arrow


In [3]:
for i, batch in enumerate(datamodule.train_dataloader()):
    print(f"batch {i}: \n{batch}")

batch 0: 
{'label_id': tensor([0, 0, 1, 1, 1, 1, 1, 1]), 'seq_len': tensor([19, 19, 20, 20, 20, 20, 21, 23]), 'token_ids': tensor([[128,  28,  52, 129, 100,   2, 130,  77,  48, 131, 132, 133,  14, 134,
         135,  28, 136, 137, 128,   0,   0,   0,   0],
        [ 80, 210, 211, 212,  30, 213,   2, 214, 176, 215,  39,  77, 216,  30,
         217, 218,  77, 175,   8,   0,   0,   0,   0],
        [ 12,  13,  14,  15,  16,  17,  18,  19,  14,  20,   2,  21,  22,  23,
           2,  24,  25,  26,  27,   8,   0,   0,   0],
        [ 28,  29,  30,  31,  30,  32,  30,  33,   4,  34,  35,  36,  37,  38,
          28,  39,   1,  40,  41,   8,   0,   0,   0],
        [ 68,  69,  70,  71,  14,  72,  73,  32,  14,  74,  28,  52,   1,  75,
          76,  77,  78,   4,  79,   8,   0,   0,   0],
        [  2, 168,  30, 169, 170, 100, 171,  16, 172, 173,  32,  30, 174,  30,
         175,  30,   4, 176, 177,   8,   0,   0,   0],
        [ 45,  46,  47,  48,   4,   2,  49,  50,  51,  30,  28,  52,  18,

In [4]:
# construct task
task = XlmrDocClassificationTask(datamodule=datamodule, num_class=2,
                                 lr=0.01)

# train model
trainer = Trainer(max_epochs=5, fast_dev_run=True)
trainer.fit(task, datamodule=datamodule)

# export to TorchScript
export_path = "/tmp/doc_task.pt1"
task.to_torchscript("/tmp/doc_task.pt1")
with open(export_path, "rb") as f:
    ts_module = torch.load(f)
    print(ts_module(text_batch=["hello world", "attend is all your need!"]))

GPU available: False, used: False
TPU available: None, using: 0 TPU cores
Running in fast_dev_run mode: will run a full train, val and test loop using 1 batch(es)
/Users/stevenliu/opt/anaconda3/envs/hack_week/lib/python3.7/site-packages/pytorch_lightning/utilities/distributed.py:49: UserWarning: you defined a validation_step but have no val_dataloader. Skipping validation loop
  warnings.warn(*args, **kwargs)

  | Name      | Type             | Params
-----------------------------------------------
0 | loss      | CrossEntropyLoss | 0     
1 | valid_acc | Accuracy         | 0     
2 | test_acc  | Accuracy         | 0     
3 | model     | RobertaModel     | 13.5 M
-----------------------------------------------
13.5 M    Trainable params
0         Non-trainable params
13.5 M    Total params
/Users/stevenliu/opt/anaconda3/envs/hack_week/lib/python3.7/site-packages/pytorch_lightning/utilities/distributed.py:49: UserWarning: The dataloader, train dataloader, does not have many workers whic

RuntimeError: 
Module 'XlmrDocClassificationTask' has no attribute 'datamodule' :
  File "/Users/stevenliu/workspace/hack_week/text/examples/xlmr_doc_classification/task.py", line 40
    def forward(self, text_batch: List[str]) -> Tensor:
        token_ids: List[Tensor] = [torch.tensor(self.datamodule.text_transform(text), dtype=torch.long) for text in text_batch]
                                                ~~~~~~~~~~~~~~~ <--- HERE
        model_inputs: Tensor = pad_sequence(token_ids, batch_first=True)
        return self.model(model_inputs)


In [ ]:
ts_module
